In [1]:
import requests
import json

url = "https://www.nordpoolgroup.com/api/marketdata/page/10"
headers = {'user-agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)
data = json.loads(response.text)


In [2]:
data.keys()

dict_keys(['data', 'cacheKey', 'conf', 'header', 'endDate', 'currency', 'pageId'])

In [3]:
data['data'].keys()


dict_keys(['Rows', 'IsDivided', 'SectionNames', 'EntityIDs', 'DataStartdate', 'DataEnddate', 'MinDateForTimeScale', 'AreaChanges', 'Units', 'LatestResultDate', 'ContainsPreliminaryValues', 'ContainsExchangeRates', 'ExchangeRateOfficial', 'ExchangeRatePreliminary', 'ExchangeUnit', 'DateUpdated', 'CombinedHeadersEnabled', 'DataType', 'TimeZoneInformation'])

In [4]:
rows = data['data']['Rows']
rows[0]


{'Columns': [{'Index': 0,
   'Scale': 2,
   'SecondaryValue': None,
   'IsDominatingDirection': False,
   'IsValid': True,
   'IsAdditionalData': False,
   'Behavior': 0,
   'Name': 'SYS',
   'Value': '4,65',
   'GroupHeader': '',
   'DisplayNegativeValueInBlue': False,
   'CombinedName': 'SYS',
   'DateTimeForData': '0001-01-01T00:00:00',
   'DisplayName': '4,65_True',
   'DisplayNameOrDominatingDirection': '4,65',
   'IsOfficial': True,
   'UseDashDisplayStyle': False},
  {'Index': 1,
   'Scale': 2,
   'SecondaryValue': None,
   'IsDominatingDirection': False,
   'IsValid': True,
   'IsAdditionalData': False,
   'Behavior': 0,
   'Name': 'SE1',
   'Value': '1,49',
   'GroupHeader': '',
   'DisplayNegativeValueInBlue': False,
   'CombinedName': 'SE1',
   'DateTimeForData': '0001-01-01T00:00:00',
   'DisplayName': '1,49_True',
   'DisplayNameOrDominatingDirection': '1,49',
   'IsOfficial': True,
   'UseDashDisplayStyle': False},
  {'Index': 2,
   'Scale': 2,
   'SecondaryValue': None,


In [17]:
import requests
import pandas as pd
from datetime import datetime

# Fetch data from the Nord Pool API
url = "https://www.nordpoolgroup.com/api/marketdata/page/10"
response = requests.get(url)
data = response.json()

# Extract the data
rows = data['data']['Rows']

# Create a dictionary to store energy prices for each region
energy_prices = {}

# Iterate through the rows
for row in rows:
    if row['GroupHeader'] is not None:
        region = row['GroupHeader']

    # Initialize a dictionary for the region if it doesn't exist
    if region not in energy_prices:
        energy_prices[region] = {}

    # Iterate through the columns
    for column in row['Columns']:
        if 'DateTimeForData' not in column:
            print("Datetime not found in column:", column)
            continue

        timestamp = datetime.strptime(column['DateTimeForData'], '%Y-%m-%dT%H:%M:%S')
        hour_range = f"{timestamp.hour:02d}&nbsp;-&nbsp;{(timestamp.hour + 1) % 24:02d}"
        value = float(column['Value'].replace(',', '.'))

        # Store the data for the region and hour
        energy_prices[region][hour_range] = value

# Convert the energy prices dictionary to a pandas DataFrame
price_data = []

for region, prices in energy_prices.items():
    for hour_range, price in prices.items():
        price_data.append({
            'Region': region,
            'Hour': hour_range,
            'Price (EUR/MWh)': price
        })

df = pd.DataFrame(price_data)

# Print the DataFrame
print(df)

# Pivot the DataFrame to get the desired table format
table = df.pivot_table(index='Hour', columns='Region', values='Price (EUR/MWh)')

# Display the table
table


KeyError: 'GroupHeader'